In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_original=pd.read_csv("data/DatosViviendas1.csv", encoding='latin-1', on_bad_lines='skip', sep=';')

C:\Users\emebe\AppData\Local\Temp\ipykernel_9240\1663228048.py:1: DtypeWarning: Columns (0,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original=pd.read_csv("data/DatosViviendas1.csv", encoding='latin-1', on_bad_lines='skip', sep=';')


In [3]:
df = df_original.copy()

In [4]:
df = df[df["NCA"]=="Cataluña"].reset_index(drop=True)

In [5]:
df['Fecha'] = pd.to_datetime(df['Fecha'])

# ordenar por id y fecha, manteniendo el mas reciente por ID
df_sorted = df.sort_values(by=['ID', 'Fecha'], ascending=[True, False])
df = df_sorted.drop_duplicates(subset='ID', keep='first').reset_index()
df.shape

C:\Users\emebe\AppData\Local\Temp\ipykernel_9240\1187895110.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Fecha'] = pd.to_datetime(df['Fecha'])


(72537, 28)

In [6]:
df.loc[0]["URL_Cliente"]

'https://www.fotocasa.es/es/inmobiliaria-m-amparo-badia-vila/comprar/inmuebles/espana/todas-las-zonas/l?clientId=9202750511730'

In [7]:
df.loc[0].T

index                                                          60453
Unnamed: 0                                                   2414107
Fecha                                            2021-09-08 00:00:00
ID                                                       116893306.0
URL                https://www.fotocasa.es/es/comprar/vivienda/vi...
ID_Cliente                                               9,20275E+12
URL_Cliente        https://www.fotocasa.es/es/inmobiliaria-m-ampa...
Inmueble                                                        Flat
Caracteristicas                                            Apartment
Habitaciones                                                     1.0
Aseos                                                            1.0
Terraza                                                          0.0
Piscina                                                          0.0
Garaje                                                           1.0
Precio                            

In [8]:
df["Caracteristicas"].value_counts()

Caracteristicas
Flat                     44027
House_Chalet             12002
Apartment                 4572
SemidetachedHouse         4003
Attic                     2285
Duplex                    1961
CountryHouse              1715
GroundFloorWithGarden     1247
SemiDetached               276
Study                      274
Loft                       158
Name: count, dtype: int64

In [9]:
#Limpiar fechas
df['Fecha']=pd.to_datetime(df['Fecha'], format='%d/%m/%Y', errors='coerce')
df = df.dropna(subset=['Fecha'])

In [10]:
#Fillna caracteristicas
df["Caracteristicas"] = df["Caracteristicas"].fillna("Unknown")

In [11]:
df['ID'] = df["ID"].astype(int).astype(str)

In [12]:
#Eliminar columnas
df.drop(["Unnamed: 0", "ID_Cliente", "URL_Cliente", "Inmueble", "URL", "Relacion",
        "CMUN", "CPRO", "CCA", "index", "Fecha", "Precision"], axis=1, inplace=True)

In [13]:
#Eliminar muestras con Habitaciones nulas
df = df.dropna(subset=['Habitaciones'])
df['Habitaciones'] = df["Habitaciones"].astype(int)

In [14]:
#Si Aseos es nulo, forzamos valor 1
df.loc[df["Aseos"].isnull(), 'Aseos'] = 1
df['Aseos'] = df["Aseos"].astype(int)

In [15]:
df["Terraza"] = df["Terraza"].astype(int)
df["Piscina"] = df["Piscina"].astype(int)
df["Garaje"] = df["Garaje"].astype(int)

In [16]:
# Eliminar muestras sin Precio
df = df.dropna(subset=['Precio'])
df['Precio'] = df["Precio"].astype(int)

In [17]:
# Eliminar muestras sin Metros
df = df.dropna(subset=['Metros'])
df["Metros"]=df["Metros"].astype(int)

In [18]:
#Arreglar latitud / longitud
def fix_coordenates(code, longitud=False):
    code2 = str(code).replace(".","")
    # Si el primer carácter no es '0' y la longitud es menor que 7
    if code2.replace("-","")[0] != '0':
        if code[0]=="-":
            return int(code2.ljust(8, '0'))/1000000
        else:
            if longitud:
                return int(code2.ljust(7, '0'))/1000000
            else:
                return int(code2.ljust(7, '0'))/100000
    else:
        return code

# Aplicar la función a la columna
df['Latitud'] = df['Latitud'].apply(fix_coordenates)
df['Longitud'] = df['Longitud'].apply(fix_coordenates, longitud=True)

In [19]:
#CUDIS
df = df.dropna(subset=['CUDIS'])
df["CUDIS"] = df["CUDIS"].astype(int).astype(str)
df['CUDIS'] = df['CUDIS'].str.zfill(7)

In [20]:
# Para las muestras sin codigo postal tenemos un mapping
cp_dict = {
    'Torrevieja': '03180',
    'Marbella': '29600',
    'Cambrils': '43850',
    'Dénia': '03700',
    'Cubelles': '08880',
    'Mercadal, Es': '07740',
    "Ametlla de Mar, L'": '43860',
    'Palamós': '17230',
    'Tarragona': '43001',
    'Benalmádena': '29631',
    'Ayamonte': '21400',
    'Badalona': '08911',
    "Castell-Platja d'Aro": '17250',
    'Tiana': '08391',
    'Vendrell, El': '43700',
    'Torredembarra': '43830',
    'Cunit': '43881',
    'Madrid': '28001',
    'Alicante/Alacant': '03001'
}

# Asignar los códigos postales usando map
df.loc[df['CodigoPostal'].isnull(), 'CodigoPostal'] = df['NMUN'].map(cp_dict)

# Eliminar los nulos
df = df.dropna(subset=['CodigoPostal'])

In [21]:
codigo_postal_nonum = pd.to_numeric(df['CodigoPostal'], errors='coerce').isna()
cp_dict = {
    'Ripoll': '17500',
    'Covaleda': '42180',
    'Graus': '22430',
    'Vilanova de Meià': '25691',
    'Mojácar': '04638',
    'Massanassa': '46470'
}

# Asignar los códigos postales solo a las filas donde CP es nulo
df.loc[codigo_postal_nonum, 'CodigoPostal'] = df['NMUN'].map(cp_dict)

In [22]:
# Formato codigo postal
df.loc[:,"CodigoPostal"] = df["CodigoPostal"].astype(int).astype(str)
df['CodigoPostal'] = df['CodigoPostal'].str.zfill(5)

In [23]:
most_common_mun = df.groupby('CodigoPostal')['NMUN'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
most_common_pro = df.groupby('CodigoPostal')['NPRO'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
most_common_cca = df.groupby('CodigoPostal')['NCA'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
most_common_cudis = df.groupby('CodigoPostal')['CUDIS'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

def replace_with_most_common(row, most_common):
    return most_common[row['CodigoPostal']]
    
df['NMUN2'] = df.apply(replace_with_most_common, most_common = most_common_mun, axis=1)
df['NPRO2'] = df.apply(replace_with_most_common, most_common = most_common_pro, axis=1)
df['NCA2'] = df.apply(replace_with_most_common, most_common = most_common_cca, axis=1)
df['CUDIS2'] = df.apply(replace_with_most_common, most_common = most_common_cudis, axis=1)

In [24]:
df = df[(df["CUDIS"].str[:-2]==df["CUDIS2"].str[:-2]) | (df["NMUN"]==df["NMUN2"]) | (df["NPRO"]==df["NPRO2"]) | (df["NCA"]==df["NCA2"])]
df = df.dropna(subset=['NMUN', 'NCA', 'NPRO', 'CUDIS'])

In [25]:
df.drop(["CUDIS2", "NMUN2", "NPRO2", "NCA2"], axis=1, inplace=True)

In [26]:
#Algunos codigos postales no hacen match con la provincia que pone que es
provincias_codigos = {
    '02': 'Albacete', '03': 'Alicante', '04': 'Almería', '01': 'Araba/Álava', '33': 'Asturias', 
    '06': 'Badajoz', '07': 'Balears, Illes', '08': 'Barcelona', '48': 'Bizkaia', '09': 'Burgos', 
    '39': 'Cantabria', '12': 'Castellón/Castelló', '51': 'Ceuta', '13': 'Ciudad Real', 
    '15': 'Coruña, A', '16': 'Cuenca', '10': 'Cáceres', '11': 'Cádiz', '14': 'Córdoba', 
    '20': 'Gipuzkoa', '17': 'Girona', '18': 'Granada', '19': 'Guadalajara', '21': 'Huelva', 
    '22': 'Huesca', '23': 'Jaén', '24': 'León', '25': 'Lleida', '27': 'Lugo', '28': 'Madrid', 
    '52': 'Melilla', '30': 'Murcia', '29': 'Málaga', '31': 'Navarra', '32': 'Ourense', 
    '34': 'Palencia', '35': 'Palmas, Las', '36': 'Pontevedra', '26': 'Rioja, La', 
    '37': 'Salamanca', '38': 'Santa Cruz de Tenerife', '40': 'Segovia', '41': 'Sevilla', 
    '42': 'Soria', '43': 'Tarragona', '44': 'Teruel', '45': 'Toledo', '46': 'Valencia/Valéncia', 
    '47': 'Valladolid', '49': 'Zamora', '50': 'Zaragoza', '05': 'Ávila'
}

In [27]:
new_pro = []
for x in df["CodigoPostal"].str[:2]:
    new_pro.append(provincias_codigos[x])
df["N_NPRO"]=new_pro

In [28]:
#Despues de comparar con las coordenadas, podemos determinar que el codigo postal es incorrecto
df = df[df["N_NPRO"]==df["NPRO"]]
df.drop(["N_NPRO"],inplace=True, axis=1)

In [29]:
df["CUDIS"]

0        2524301
1        4312303
2        4313602
3        4309201
4        4314701
          ...   
72532    0811302
72533    0814101
72534    4314806
72535    0818701
72536    0808901
Name: CUDIS, Length: 69408, dtype: object

In [31]:
#Precio del metro cuadrado
df["PrecioM2"]=df["Precio"]/df["Metros"]

In [32]:
df = df[df['Caracteristicas'] != 'Unknown']

# Mapping de caracteristicas
df['Caracteristicas'] = df['Caracteristicas'].replace({
    'Flat': 'Apartment',
    'Loft': 'Apartment',
    'Study': 'Apartment',
    'GroundFloorWithGarden': 'Apartment',
    'Attic': 'Apartment',
    'Duplex': 'Apartment',
    'Apartment': 'Apartment',
    'House_Chalet': 'House',
    'SemidetachedHouse': 'House',
    'SemiDetached': 'House'
})

In [33]:
df.to_csv("output/clean_dataset.csv", index=False)

In [34]:
df.shape

(69394, 17)

In [35]:
df_original.shape

(954157, 27)

In [36]:
df.columns

Index(['ID', 'Caracteristicas', 'Habitaciones', 'Aseos', 'Terraza', 'Piscina',
       'Garaje', 'Precio', 'Metros', 'CodigoPostal', 'Latitud', 'Longitud',
       'CUDIS', 'NPRO', 'NCA', 'NMUN', 'PrecioM2'],
      dtype='object')